In [ ]:
import numpy as np
import pandas as pd
import string
import random

In [ ]:
spam=[] 
corpus=[]
train=[]
test=[]
train_spam=[]
test_spam=[]

corpus_dict=set()
inv_index={}

prob_spam=[]
prob_ham=[]

In [ ]:
def initialize():
    global test,train,train_spam,test_spam,corpus_dict,inv_index,prob_spam,prob_ham
    train=[]
    test=[]
    train_spam=[]
    test_spam=[]
    corpus_dict=set()
    inv_index={}
    prob_spam=[]
    prob_ham=[]

In [ ]:
#remove stopwords and punctuations
def process_text(text):
  nopunc=[char for char in text if char not in string.punctuation]
  nopunc=''.join(nopunc)
  stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
  clean_words=[word.lower() for word in nopunc.split() if word.lower() not in stop_words]
  #print(clean_words)
  return clean_words

In [ ]:
def k_fold(k):
    global corpus,spam,train,test,train_spam,test_spam
    
    #update train and test spam
    for s in spam[k]:
        test_spam.append(s)
    for j in range(7):
        if k!=j:
            for s in spam[j]:
                train_spam.append(s)
    #update train and test
    for sentence in corpus[k]:
        test.append(sentence)
    for j in range(7):
        if k!=j:
            for sentence in corpus[j]:
                train.append(sentence)
    #print(corpus)
    #print(spam)
    #print(train)
    #print(test)
    #print(train_spam)
    #print(test_spam)

In [ ]:
def make_dict():
    global corpus_dict,train
    corpus_dict=set()
    for sentence in train:
        for word in sentence:
            corpus_dict.add(word)
    #print(corpus_dict)

In [ ]:
def build_inv_index():
    global corpus_dict,train,inv_index
    inv_index = {i:[] for i in corpus_dict}
    #print(type(inv_index))
    for idx, text in enumerate(train):
        for word in text:
            inv_index[word].append(idx)
    #print(inv_index)

In [ ]:
def corpus_prob():
    global prob_spam,prob_ham,inv_index,train_spam,corpus_dict
    total_spam=sum(train_spam)
    total_ham=len(train_spam)-total_spam
    for word in corpus_dict:
        no_of_spam=0
        no_of_ham=0
        #print(inv_index[word])
        for idx in inv_index[word]:
            #print(idx)
            #print(len(train_spam))
            if train_spam[idx]==1:
                no_of_spam+=1
            else:
                no_of_ham+=1
        curr_spam_prob=no_of_spam/total_spam
        curr_ham_prob=no_of_ham/total_ham
        prob_spam.append(curr_spam_prob)
        prob_ham.append(curr_ham_prob)

In [ ]:
def find_prob(clean_words,total_spam_prob,total_ham_prob):
    global prob_spam,prob_ham,train_spam
    no_of_spam=sum(train_spam)
    no_of_ham=len(train_spam)-no_of_spam
    spam_test=no_of_spam/(no_of_spam+no_of_ham)
    ham_test=no_of_spam/(no_of_spam+no_of_ham)
    for word in clean_words:
        curr_spam=0
        curr_ham=0
        if word in inv_index:
            for id in inv_index[word]:
                if train_spam[id]==1:
                    curr_spam+=1
                else:
                    curr_ham+=1
        prob_spam_curr=(curr_spam+1)/(no_of_spam+2)
        prob_ham_curr=(curr_ham+1)/(no_of_ham+2)
        spam_test*=prob_spam_curr
        ham_test*=prob_ham_curr
    if spam_test>ham_test:
        return 1
    else:
        return 0

In [ ]:
def test_prob():
    global test,train_spam,test_spam
    total_spam_prob=sum(train_spam)/len(train_spam)
    total_ham_prob=1-total_spam_prob
    correct=0
    total=0
    for id,sentence in enumerate(test):
        is_spam=find_prob(sentence,total_spam_prob,total_ham_prob)
        if is_spam==test_spam[id]:
            correct+=1
        total+=1
    return correct/total

In [ ]:
#main

f=open("/content/drive/MyDrive/Naive bayes/dataset_NB.txt","r")

for line in f:
    words=line.split()
    words_without_num=words[:-1]  #removing the spam value in the line
    line_formed=' '.join([str(elem) for elem in words_without_num]) #joining the rest of the words for line

    clean_words=process_text(line_formed) #remove stopwords and 1-2 letter words
    s=set(clean_words)
    corpus.append(s)  

    is_spam=words[-1] 
    spam.append(int(is_spam)) #add the spam value to the global spam vector

#shuffle
temp=list(zip(corpus,spam))
random.seed(0)
random.shuffle(temp)
corpus1,spam1=zip(*temp)

#split in k-fold
split_list=[143,286,429,572,715,858]
corpus_mod=[corpus1[i:j] for i,j in zip([0] + split_list, split_list + [None])]
spam_mod=[spam1[i:j] for i,j in zip([0] + split_list, split_list + [None])]
corpus=corpus_mod
spam=spam_mod

sum_of_eff=0
for i in range(7):
    initialize()
    k_fold(i)
    make_dict()
    build_inv_index()
    corpus_prob()
    eff=test_prob()
    sum_of_eff+=eff
    print("Accuracy at",i,"th fold is",eff)
print("Overall accuracy: ",sum_of_eff/7)

Accuracy at 0 th fold is 0.7832167832167832
Accuracy at 1 th fold is 0.8181818181818182
Accuracy at 2 th fold is 0.8391608391608392
Accuracy at 3 th fold is 0.7832167832167832
Accuracy at 4 th fold is 0.7762237762237763
Accuracy at 5 th fold is 0.7692307692307693
Accuracy at 6 th fold is 0.8309859154929577
Overall accuracy:  0.8000309549605324
